## Feature Extraction
(from Openface data)

[Notebook to download youtube videos](https://colab.research.google.com/drive/1aAIrmdMbz8Z9w3UT3n7aUBCQ5BWKP6al)

[Notebook to get openface output](https://colab.research.google.com/drive/1_YtiBCoB4-HEQm0M2rNqgAfYXdZfB3FK)

In [ ]:
#PROJECT_ROOT = '/content/drive/My Drive/school-419-project'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')
#%cd {PROJECT_ROOT}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/school-419-project


In [ ]:
!pip install tslearn

     |████████████████████████████████| 778kB 8.1MB/s 


In [1]:
import numpy as np
import os
import pandas as pd#; pd.set_option('display.max_colwidth', None)
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from pathlib import Path

In [ ]:
!ls -al | head -n 5

## Load Openface CSVs

In [3]:
savename = "aggregated_openface_data.pickle"

openface_csvs = "./openface_processed/multi"

df = pd.DataFrame()

if os.path.isfile(savename):
  print("Loading combined CSV's from file")
  df = pd.read_pickle(savename)
else:
  for root, dirs, files in os.walk(openface_csvs):
    dirs.clear()  # don't recurse

    for f in [f for f in files if f[-4:] == '.csv']:
      tempdata = pd.read_csv(os.path.join(root, f), skipinitialspace=True)
      tempdata["filename"] = f[0:-4].strip()  # make column with sample number (remove .csv suffix, whitespace)
      df = pd.concat([df, tempdata])

  print("Saving combined CSV's to file")
  df.to_pickle(savename)
df.describe()

Loading combined CSV's from file


,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
count,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,...,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000
mean,155.616133,0.501809,4.605910,0.751813,0.766122,0.036090,0.203211,-0.663631,-0.076915,0.208771,...,0.294790,0.399913,0.581600,0.286767,0.381905,0.208197,0.404229,0.323261,0.406584,0.318552
std,164.296348,0.710695,4.882572,0.367902,0.423305,0.332307,0.171565,0.378508,0.330183,0.175701,...,0.455958,0.489891,0.493307,0.452262,0.485864,0.406027,0.490753,0.467732,0.491207,0.465925
min,1.000000,0.000000,0.000000,0.000000,0.000000,-0.805486,-0.448546,-0.999994,-0.914196,-0.414886,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,46.000000,0.000000,1.600000,0.770000,1.000000,-0.196749,0.000000,-0.932839,-0.361876,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,97.000000,0.000000,3.365000,0.980000,1.000000,0.000000,0.210941,-0.857065,0.000000,0.219472,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,188.000000,1.000000,5.772000,0.980000,1.000000,0.286586,0.324906,-0.578735,0.125412,0.340716,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,837.000000,3.000000,34.868000,0.980000,1.000000,0.931122,0.726961,0.000000,0.893593,0.843826,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


There are hundreds of columns, let's only consider gaze locations, angles, poses, and AU's with high confidence

In [4]:
df = pd.concat([df.loc[:,"filename"], df.loc[:,"frame":"gaze_angle_y"], df.loc[:,"pose_Tx":"pose_Rz"], df.loc[:,"AU01_r":"AU45_c"]],
                axis=1)
df = df[df["confidence"] > .75]
df.describe()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
count,17571.000000,17571.000000,17571.000000,17571.000000,17571.0,17571.000000,17571.000000,17571.000000,17571.000000,17571.000000,...,17571.000000,17571.000000,17571.000000,17571.000000,17571.000000,17571.000000,17571.000000,17571.000000,17571.000000,17571.000000
mean,153.537135,0.448295,4.743672,0.946658,1.0,0.047107,0.265246,-0.866222,-0.100395,0.272504,...,0.303910,0.430425,0.545843,0.238063,0.361732,0.183541,0.410961,0.375221,0.375448,0.371464
std,159.454405,0.672569,5.120916,0.052381,0.0,0.378974,0.148206,0.107279,0.374094,0.151417,...,0.459957,0.495150,0.497908,0.425910,0.480516,0.387120,0.492022,0.484194,0.484252,0.483210
min,1.000000,0.000000,0.000000,0.770000,1.0,-0.805486,-0.448546,-0.999994,-0.914196,-0.414886,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,1.668000,0.880000,1.0,-0.278707,0.175896,-0.947493,-0.415882,0.183876,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,98.000000,0.000000,3.433000,0.980000,1.0,0.072186,0.259811,-0.894062,-0.105439,0.271059,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,186.000000,1.000000,5.839000,0.980000,1.0,0.347410,0.357944,-0.814608,0.196633,0.368472,...,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,831.000000,3.000000,34.618000,0.980000,1.0,0.931122,0.726961,-0.225817,0.893593,0.843826,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
print(np.shape(df))
df.head(5)

(17571, 55)


,filename,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,flirt-qkq9k5FewvI-1,1,0,0.000,0.93,1,-0.154035,0.282251,-0.946894,-0.317694,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,flirt-qkq9k5FewvI-1,2,0,0.033,0.93,1,-0.157176,0.285725,-0.945334,-0.320966,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,flirt-qkq9k5FewvI-1,3,0,0.067,0.93,1,-0.156784,0.289411,-0.944278,-0.321495,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,flirt-qkq9k5FewvI-1,4,0,0.100,0.93,1,-0.156940,0.289137,-0.944335,-0.322308,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,flirt-qkq9k5FewvI-1,5,0,0.133,0.93,1,-0.155533,0.286694,-0.945313,-0.320138,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Reshape into tslearn format

"If time series from the set are not equal-sized, NaN values are appended to the shorter ones and the shape of the resulting array is:

`(n_ts, max_sz, d)`, where `max_sz` is the maximum of sizes for time series in the set." - [tslearn docs](https://tslearn.readthedocs.io/en/stable/gettingstarted.html#time-series-format)

(`n_ts` $\rightarrow$ number of sequences, `d` $\rightarrow$ the features.)

In [7]:
[data.shape for filename,data in df.groupby("filename")][0:5]  # number of samples, number of features

[(598, 55), (114, 55), (223, 55), (289, 55), (301, 55)]

In [ ]:
#{filename: data.drop('filename', axis=1) for filename,data in df.groupby("filename")}

In [ ]:
from tslearn.utils import to_time_series_dataset

dataset = to_time_series_dataset([data.drop('filename', axis=1) for filename,data in df.groupby("filename")])
dataset

array([[[1.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 0.000e+00,
         1.000e+00],
        [2.000e+00, 0.000e+00, 4.200e-02, ..., 1.000e+00, 0.000e+00,
         1.000e+00],
        [3.000e+00, 0.000e+00, 8.300e-02, ..., 1.000e+00, 0.000e+00,
         1.000e+00],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[1.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.000e+00, 0.000e+00, 4.200e-02, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [3.000e+00, 0.000e+00, 8.300e-02, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
   

In [ ]:
np.shape(df.groupby("filename"))

(20, 2)

In [ ]:
dataset.shape  # n_timeseries (files), max frames, features

(20, 802, 54)

In [ ]:
!pip install neural_structured_learning

     |████████████████████████████████| 122kB 7.9MB/s 


In [ ]:
import tensorflow as tf
import neural_structured_learning as nsl

# Prepare data.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#tf.keras.layers.Embedding

# Create a base model -- sequential, functional, or subclass.
model = tf.keras.Sequential([
    tf.keras.Input((28, 28), name='feature'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Wrap the model with adversarial regularization.
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)

# Compile, train, and evaluate.
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
adv_model.fit({'feature': x_train, 'label': y_train}, batch_size=32, epochs=5)
adv_model.evaluate({'feature': x_test, 'label': y_test})


11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f7150622c00> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f7150622c00> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f7150622c00> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


1875/1875 [==============================] - 6s 3ms/step - loss: 0.3153 - sparse_categorical_crossentropy: 0.2578 - sparse_categorical_accuracy: 0.9251 - scaled_adversarial_loss: 0.0575
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1421 - sparse_categorical_crossentropy: 0.1148 - sparse_categorical_accuracy: 0.9664 - scaled_adversarial_loss: 0.0272
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0993 - sparse_categorical_crossentropy: 0.0797 - sparse_categorical_accuracy: 0.9760 - scaled_adversarial_loss: 0.0196
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0751 - sparse_categorical_crossentropy: 0.0599 - sparse_categorical_accuracy: 0.9821 - scaled_adversarial_loss: 0.0152
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0576 - sparse_categorical_crossentropy: 0.0456 - sparse_categorical_accuracy: 0.9858 - scaled_adversarial_loss: 0.0120


313/313 [==============================] - 1s 2ms/step - loss: 0.0989 - sparse_categorical_crossentropy: 0.0791 - sparse_categorical_accuracy: 0.9761 - scaled_adversarial_loss: 0.0198


[0.09892771393060684,
 0.07911036163568497,
 0.9761000275611877,
 0.019817402586340904]

## Literally copied from https://github.com/ml-jku/hopfield-layers/blob/da336c7b0ceef8dd313c687f760b0b6d6df76848/examples/bit_pattern_demo.ipynb
## edit this! $\downarrow$


In [ ]:
# Import general modules used e.g. for plotting.
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import sys
import torch

# Importing Hopfield-specific modules.
from auxiliary.data import BitPatternSet
from modules import Hopfield, HopfieldPooling

# Import auxiliary modules.
from distutils.version import LooseVersion
from typing import List, Tuple

# Importing PyTorch specific modules.
from torch import Tensor
from torch.nn import Flatten, Linear, Module, Sequential
from torch.nn.functional import binary_cross_entropy_with_logits
from torch.nn.utils import clip_grad_norm_
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

ModuleNotFoundError: ignored

In [ ]:
device = torch.device(r'cuda:0' if torch.cuda.is_available() else r'cpu')

# Create data loader of training set.
sampler_train = SubsetRandomSampler(list(range(256, 2048 - 256)))
data_loader_train = DataLoader(dataset=bit_pattern_set, batch_size=32, sampler=sampler_train)

# Create data loader of validation set.
sampler_eval = SubsetRandomSampler(list(range(256)) + list(range(2048 - 256, 2048)))
data_loader_eval = DataLoader(dataset=bit_pattern_set, batch_size=32, sampler=sampler_eval)

NameError: ignored

In [ ]:
import midi
import pandas as pd
import tensorflow as tf
import time

from pathlib import Path
OPENFACE_CSV_LOCATION = "openface_processed"

class VideoOpenfaceDataset(tf.data.Dataset):
    @staticmethod
    def _generator(train_or_test: str):
        data_folder = Path(OPENFACE_CSV_LOCATION)
        ourPickle.unpickle()

        for f in midi_files:
            # Reading data (line, record) from the file
            yield somekinda_sample_info

    def __new__(cls, train_or_test: str):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=tf.dtypes.int64,
            output_shapes=(None, 78, 2),  # note_state_matrix size
            args=(train_or_test,)
        )

def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        print(f"Epoch: {epoch_num}")
        count=0
        for sample in dataset:
            count += 1
            print(f"Sample: {count}")
            # Performing a training step
            time.sleep(0.01)
    print("Execution time:", time.perf_counter() - start_time)

if __name__=="__main__":
    benchmark(MidiDataset())

ModuleNotFoundError: ignored

# Cluster input clips - probably not using this, ignore

In [ ]:
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from collections import defaultdict
from tqdm import tqdm
import pickle

ts_seed = 85510309
ss_seed = 10474245


start, stop = 5, 15

savename = f"kmeans_and_silhouette_score_from_{start}_to_{stop}.pickle"

score_per_run = defaultdict(list)

if os.path.isfile(savename):
  print(f"Loading saved k-means dict and silhouette scores from {save_location}")
  with open(savename, 'rb') as f:
    score_per_run = pickle.load(f)
else:
  for i in tqdm(range(start, stop)):
    km = TimeSeriesKMeans(n_clusters=i, metric="dtw", random_state=ts_seed)
    km.fit(dataset)
    ss = silhouette_score(dataset, labels=km.labels_, metric='dtw', random_state=ss_seed)

    score_per_run[i] = [ss, km]

  print(f"Saving {savename}")
  with open(savename, 'wb') as f:
    pickle.dump(score_per_run, f)

100%|██████████| 10/10 [07:56<00:00, 47.66s/it]


Saving kmeans_and_silhouette_score_from_5_to_15.pickle


We want the best-defined clusters, which is what the highest silhouette score corresponds to

In [ ]:
#num_categories = next(iter(score_per_run.keys()))
#score_per_run

[0.16582300466566252,
 TimeSeriesKMeans(dtw_inertia=False, init='k-means++', max_iter=50,
                  max_iter_barycenter=100, metric='dtw', metric_params=None,
                  n_clusters=5, n_init=1, n_jobs=None, random_state=85510309,
                  tol=1e-06, verbose=0)]

In [ ]:
score_per_run

defaultdict(list,
            {5: [0.16582300466566252,
              TimeSeriesKMeans(dtw_inertia=False, init='k-means++', max_iter=50,
                               max_iter_barycenter=100, metric='dtw', metric_params=None,
                               n_clusters=5, n_init=1, n_jobs=None, random_state=85510309,
                               tol=1e-06, verbose=0)],
             6: [0.19760275368115737,
              TimeSeriesKMeans(dtw_inertia=False, init='k-means++', max_iter=50,
                               max_iter_barycenter=100, metric='dtw', metric_params=None,
                               n_clusters=6, n_init=1, n_jobs=None, random_state=85510309,
                               tol=1e-06, verbose=0)],
             7: [0.18499278071672243,
              TimeSeriesKMeans(dtw_inertia=False, init='k-means++', max_iter=50,
                               max_iter_barycenter=100, metric='dtw', metric_params=None,
                               n_clusters=7, n_init=1, n_jobs=N

In [ ]:
print(sorted([3,1,7]))  # sort is ascending
s = sorted(score_per_run, key=lambda k: score_per_run[k][0])  # sort by silhouette score
print(s)
print(f"Best (smallest) silhouette score is with {s[0]} categories")

num_metrics = s[0]

[1, 3, 7]
[12, 14, 13, 5, 11, 7, 6, 9, 8, 10]
Best (smallest) silhouette score is with 12 categories


# Only worked on until here, it's Dylan's vanilla A3 stuff past here
## 


In [ ]:
s_score, clustered = score_per_run[num_metrics]
s_score, clustered

(0.10833492830264713,
 TimeSeriesKMeans(dtw_inertia=False, init='k-means++', max_iter=50,
                  max_iter_barycenter=100, metric='dtw', metric_params=None,
                  n_clusters=12, n_init=1, n_jobs=None, random_state=85510309,
                  tol=1e-06, verbose=0))

In [ ]:
clustered.cluster_centers_.shape  # categories, max samples, features

(12, 802, 49)

In [ ]:
clustered.labels_

array([ 8,  6,  7,  7,  3,  4,  8,  8, 11, 11, 11,  2,  8,  0,  9,  5,  8,
       10,  8,  1])

In [ ]:
from tslearn.metrics import dtw

# find the distance from each point to the center of the cluster

dists = defaultdict(float)

for fname,label in enumerate(clustered.labels_):
  timeseries = dataset[fname,:,:]
  center = clustered.cluster_centers_[label,:,:]

  dists[fname] = dtw(timeseries, center)

In [ ]:
# want to get list of gifs closest to the center of the categories

per_cluster_distances = defaultdict(list)

for fname,label in enumerate(clustered.labels_):  # one label per file
  per_cluster_distances[label].append((fname, dists[fname]))

# have a dict of categories, 

In [ ]:
per_cluster_distances[0][:10] # all the filenames belonging to cluster 0, and their associated dtw distance from center of cluster

[(13, 0.0)]

In [ ]:
sorted_cluster_distances = defaultdict(tuple)
for label in per_cluster_distances:
  sorted_cluster_distances[label] = sorted(per_cluster_distances[label], 
                                           key=lambda x: x[1])
  
# now the first entry in each cluster point list is the one with lowest distance (dtw)
sorted_cluster_distances[0][:10]

[(13, 0.0)]

# Display GIFs in cluster

In [ ]:
# sourced from https://github.com/ipython/ipython/issues/10045

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display

In [ ]:
for label in sorted(sorted_cluster_distances):
  print(f"Top two clips for label: {label}")
  for fname,score in sorted_cluster_distances[label][:5]:
    pth = os.path.join("clipped_videos", str(fname))
    print(f"Trying {pth}")
    with open(pth,'rb') as f:
      display.Image(data=f.read(), format='png', height=100)

  input("Press enter to see next set")
  display.clear_output()
  

Top two clips for label: 0
Trying clipped_videos/13


FileNotFoundError: ignored

My notes on the above categorizations:

0
more animated, scrunched eyes + horizontal movement

1
Fists in air

2
tilted head, open mouth smile, up-to-down head motions


3
wide open mouth smile, head bobbing


4
open mouth with tongue sticking out, blinking


5
head turn side to side, squinted eyes